In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier


In [2]:
import pandas_gbq
from google.oauth2 import service_account

KEYFILE = 'ensai-2025-fc6ddbb72db7.json'
credentials=service_account.Credentials.from_service_account_file(KEYFILE)

In [3]:
prenom=pandas_gbq.read_gbq('SELECT * FROM ml.prenoms LIMIT 5000', project_id='ensai-2025', credentials=credentials)

/home/ensai/Bureau/Devops/TP1/env/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2379: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


In [6]:
prenom.head()

,sexe,preusuel,annais,dpt,nombre
0,1,ARMAND,1900,75,71
1,1,GEORGES,1900,55,59
2,1,JACQUES,1900,29,47
3,1,PAUL,1900,12,77
4,1,CELESTIN,1900,23,3


In [7]:
prenom=prenom[["preusuel","sexe"]]

In [ ]:
prenom_train,prenom_test = train_test_split(prenom, test_size=0.2, random_state=42)
prenom_X,y=prenom["preusuel"],prenom["sexe"]
X_train,y_train=prenom_train["preusuel"],prenom_train["sexe"]
X_test,y_test=prenom_test["preusuel"],prenom_test["sexe"]

In [24]:
# Encodage des prénoms d'entraînement avec des variables fictives
X_train_encoded_dummies = pd.get_dummies(X_train)

# Encodage des prénoms de test avec des variables fictives
X_test_encoded_dummies = pd.get_dummies(X_test)

# Assurer que les colonnes de X_test_encoded_dummies correspondent à celles de X_train_encoded_dummies
X_test_encoded_dummies = X_test_encoded_dummies.reindex(columns=X_train_encoded_dummies.columns, fill_value=0)

In [25]:
X_train_encoded_dummies = X_train_encoded_dummies.astype(int)
X_test_encoded_dummies = X_test_encoded_dummies.astype(int)

print(X_train_encoded_dummies.shape)
print(X_test_encoded_dummies.shape)

(4000, 614)
(1000, 614)


In [29]:
model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train_encoded_dummies, y_train)


# Étape 2 : Faire des prédictions sur le jeu de test
y_pred = model.predict(X_test_encoded_dummies)

# Étape 3 : Évaluer les performances du modèle
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy sur le jeu de test : {accuracy:.2f}")

# Étape 4 : Afficher le rapport de classification
report = classification_report(y_test, y_pred)
print("Rapport de classification :\n", report)

Accuracy sur le jeu de test : 0.96
Rapport de classification :
               precision    recall  f1-score   support

           1       0.92      0.99      0.95       455
           2       0.99      0.93      0.96       545

    accuracy                           0.96      1000
   macro avg       0.96      0.96      0.96      1000
weighted avg       0.96      0.96      0.96      1000



In [30]:
# Importer les bibliothèques nécessaires
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Étape 1 : Créer et entraîner le modèle
model = RandomForestClassifier(random_state=42,n_estimators=200)  # Vous pouvez choisir d'autres modèles si besoin
model.fit(X_train_encoded_dummies, y_train)

# Étape 2 : Faire des prédictions sur le jeu de test
y_pred = model.predict(X_test_encoded_dummies)

# Étape 3 : Évaluer les performances du modèle
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy sur le jeu de test : {accuracy:.2f}")

# Étape 4 : Afficher le rapport de classification
report = classification_report(y_test, y_pred)
print("Rapport de classification :\n", report)


Accuracy sur le jeu de test : 0.95
Rapport de classification :
               precision    recall  f1-score   support

           1       0.97      0.93      0.95       455
           2       0.94      0.97      0.96       545

    accuracy                           0.95      1000
   macro avg       0.96      0.95      0.95      1000
weighted avg       0.95      0.95      0.95      1000



In [31]:
import joblib

# Sauvegarder le modèle RandomForestClassifier
joblib.dump(model, 'random_forest_model.pkl')

['random_forest_model.pkl']

In [44]:
import pandas as pd

def predict_gender(prenom, encoder, model, columns):


# Charger le modèle préalablement sauvegardé
    model = joblib.load('/home/ensai/Bureau/Devops/TP1/random_forest_model.pkl')
    # Créer un DataFrame avec le prénom
    df = pd.DataFrame([prenom], columns=['preusuel'])
    
    # Encoder le prénom avec des variables fictives
    df_encoded = pd.get_dummies(df['preusuel'])
    
    # Assurer que les colonnes de df_encoded correspondent à celles de l'encodeur d'origine
    df_encoded = df_encoded.reindex(columns=columns, fill_value=0)
    
    # Faire la prédiction
    prediction = model.predict(df_encoded)
    
    # Retourner le sexe prédit
    if prediction[0] == "1":
        return "masculin"
    else:
        return "féminin"

# Exemple d'utilisation
prenom_a_predire = 'JEAN'
sexe_pred = predict_gender(prenom_a_predire, encoder, model, X_train_encoded_dummies.columns)
print(f"Le sexe prédit pour le prénom {prenom_a_predire} est : {sexe_pred}")


Le sexe prédit pour le prénom JEAN est : masculin
